In [1]:
import ee
import datetime
import os
import itertools
import sys
import re

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

In [3]:
# define basic parameters
year_span = [f'{i}_{i+2}' for i in range(1990,2020,3)]

In [11]:
region_cn = '中南'
region_en = 'zhongnan'

##### Get the priliminary classifications

In [4]:
# prepare the different input band types
in_band_selection =[#'Spectrum',
                    #'Spectrum_Normalize',
                    #'Spectrum_Normalize_Fourier',
                    #'Spectrum_Normalize_Fourier_Terrain',
                    'Spectrum_Normalize_Fourier_Terrain_Meterology']

In [15]:
# get the priliminary classifications and select those with a 
# >5 value in the 10-folds sum, here the 5 threshold comes from
# the Step_1 10-folds check of this fold

path = 'users/rlyon2/China_built_classification/01_primary_classifications'

classification_ten_folds_check = {}

for in_band in in_band_selection:
    for year in year_span:
        
        img_name = f'{in_band}_{region_en}_{year}'
    
        preliminary_classification = ee.ImageCollection([ee.Image(f'{path}/{img_name}_{seed}') for seed in range(10)]).sum()
        img_check = preliminary_classification.gt(4)

        classification_ten_folds_check[(in_band,year)] = [img_check]
        

In [9]:
# store the classification in a df for conviniency
classification_ten_folds_check_df = pd.DataFrame(classification_ten_folds_check)

# format the df
classification_checked = classification_ten_folds_check_df.T.reset_index()
classification_checked.columns = ['In bands','year','classification']

##### Export the result

In [12]:
# import north_china_plain boundary
Region =  ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/China_zone")\
            .filterMetadata('NAME1','equals',region_cn)

In [14]:
classification_checked

,In bands,year,classification
0,Spectrum_Normalize_Fourier_Terrain_Meterology,1990_1992,"ee.Image({\n ""functionInvocationValue"": {\n ..."
1,Spectrum_Normalize_Fourier_Terrain_Meterology,1993_1995,"ee.Image({\n ""functionInvocationValue"": {\n ..."
2,Spectrum_Normalize_Fourier_Terrain_Meterology,1996_1998,"ee.Image({\n ""functionInvocationValue"": {\n ..."
3,Spectrum_Normalize_Fourier_Terrain_Meterology,1999_2001,"ee.Image({\n ""functionInvocationValue"": {\n ..."
4,Spectrum_Normalize_Fourier_Terrain_Meterology,2002_2004,"ee.Image({\n ""functionInvocationValue"": {\n ..."
5,Spectrum_Normalize_Fourier_Terrain_Meterology,2005_2007,"ee.Image({\n ""functionInvocationValue"": {\n ..."
6,Spectrum_Normalize_Fourier_Terrain_Meterology,2008_2010,"ee.Image({\n ""functionInvocationValue"": {\n ..."
7,Spectrum_Normalize_Fourier_Terrain_Meterology,2011_2013,"ee.Image({\n ""functionInvocationValue"": {\n ..."
8,Spectrum_Normalize_Fourier_Terrain_Meterology,2014_2016,"ee.Image({\n ""functionInvocationValue"": {\n ..."
9,Spectrum_Normalize_Fourier_Terrain_Meterology,2017_2019,"ee.Image({\n ""functionInvocationValue"": {\n ..."


In [20]:
for idx,row in classification_checked.iterrows():
    
    out_path = 'users/rlyon2/China_built_classification/02_ten_folds_check'
    
    # get data
    in_bands = row[0]
    year = row[1]
    img = row[2]
    
    # construct export name
    name = f'{in_bands}_{region_en}_{year}'
    
    # export
    task = ee.batch.Export.image.toAsset(    
                                            image= img,
                                            description=name,
                                            assetId=f'{out_path}/{name}',
                                            region=Region.geometry().bounds(),
                                            scale=30,
                                            maxPixels=int(1e13)
    )
    
    task.start()
    
    # print out the process
    print(f'{name} has been exported!')

Spectrum_1990_1992 has been exported!
Spectrum_1993_1995 has been exported!
Spectrum_1996_1998 has been exported!
Spectrum_1999_2001 has been exported!
Spectrum_2002_2004 has been exported!
Spectrum_2005_2007 has been exported!
Spectrum_2008_2010 has been exported!
Spectrum_2011_2013 has been exported!
Spectrum_2014_2016 has been exported!
Spectrum_2017_2019 has been exported!
Spectrum_Normalize_1990_1992 has been exported!
Spectrum_Normalize_1993_1995 has been exported!
Spectrum_Normalize_1996_1998 has been exported!
Spectrum_Normalize_1999_2001 has been exported!
Spectrum_Normalize_2002_2004 has been exported!
Spectrum_Normalize_2005_2007 has been exported!
Spectrum_Normalize_2008_2010 has been exported!
Spectrum_Normalize_2011_2013 has been exported!
Spectrum_Normalize_2014_2016 has been exported!
Spectrum_Normalize_2017_2019 has been exported!
Spectrum_Normalize_Fourier_1990_1992 has been exported!
Spectrum_Normalize_Fourier_1993_1995 has been exported!
Spectrum_Normalize_Fourier_1